In [ ]:
#Importando as bibliotecas necessárias
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
from bs4 import BeautifulSoup
import time
from datetime import date

In [ ]:
# Armazendo a data atual e transformando-a em string
data_atual = date.today()
data = str(data_atual)
    
#Lendo a planilha de entrada
df = pd.read_excel('pedidos.xlsx')
    
#Transformando a primeira coluna em uma lista
numeros = df.iloc[:, 0].tolist()
    
#Função para clicar um link específico dentro da página
def clickLink():
    navegador.find_element(By.CSS_SELECTOR, "a[href*='/ConfirmacaoPedido.asp?Pedido=']").click()
    
#Iniciando o web driver e recebendo o link da página alvo (abre o navegador)
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.get('--------------') #link do site ( utilizado aqui para acessar a página de login)
    
#Encontrando e preenchendo o campo de login no site alvo (retirados por ser privado)
navegador.find_element(By.XPATH, '//*[@id="LoginName"]').send_keys('----------') 
navegador.find_element(By.XPATH, '//*[@id="RAWSenha"]').send_keys('----------')
navegador.find_element(By.XPATH, '//*[@id="SubmitAuth"]').click()

In [ ]:
#Criando a planilha de output
workbook = Workbook()
worksheet = workbook.active
headers = ["Pedido", "CNPJ", "Endereço","Município","UF","CEP"]  # Replace with your actual headers
for col_num, header in enumerate(headers, start=1):
    worksheet.cell(row=1, column=col_num, value=header)
        
linha = 2

In [ ]:
#iterando sobre a coluna transformada em lista
for pedido in numeros:
    #Todos os time.sleep são pra garantir que o html carregue a tempo
    time.sleep(5)
    #Navegando na página
    navegador.find_element(By.XPATH,'//*[@id="inbox-grid"]/header/div/div/div/div[1]/div[2]/button[1]').click()
    time.sleep(2)
    navegador.find_element(By.XPATH,"//div[@class='dropdown b-dropdown']//button[contains(@class, 'btn-outline-primary')]").click()
    navegador.find_element(By.XPATH, "//input[contains(@class, 'form-control form-control-md') and contains(@placeholder, 'Buscar filtro')]").send_keys('Número do Pedido')
    
    #Ação de clicar a setinha pra baixo e em seguida dar enter
    actions = ActionChains(navegador)
    actions.send_keys(Keys.DOWN)
    actions.send_keys(Keys.ENTER)
    actions.perform()
    
    #Navegando na página
    navegador.find_element(By.XPATH, "//input[contains(@class, 'form-control form-control-sm')]").send_keys(pedido)
    navegador.find_element(By.XPATH,'//*[@id="inbox-grid"]/header/div/div/div/div[2]/div[1]/div[3]/button[2]').click()
    #####################################################################
    time.sleep(1)
    clickLink()
        
    #Coletando as informações necessárias utilizando o Beautiful Soup e colocando elas na planilha de output
    
    info_pedido = pedido
    worksheet.cell(row=linha, column=1, value=info_pedido)
    
    #html = navegador.find_element(By.XPATH,"//div[@class='modal-body doc-modal__body p-0 no-header-border no-footer-border']")
    
    iframe = navegador.find_element(By.ID, "document-details-iframe")
    navegador.switch_to.frame(iframe)
    
    html = navegador.page_source

    soup = BeautifulSoup(html, 'html.parser')
    
    text_cnpj = soup.find('b', string='CNPJ :')
    
    #Condicional para pular a linha caso não encontre o campo CNPJ
    if text_cnpj == None:
        #print('eu')
        linha += 1
        actions = ActionChains(navegador)
        actions.send_keys(Keys.ESCAPE)
        actions.perform()
        navegador.get("--------------") #link do site (utilizado aqui para voltar para homepage)
        continue
    
    info_cnpj = text_cnpj.find_next_sibling(string=True).strip()
    
    #CNPJ
    worksheet.cell(row=linha, column=2, value=info_cnpj)
    end = soup.find_all('br')
    end_info = end[14].find_next_sibling(string=True).strip('-')
    
    #endereço
    worksheet.cell(row=linha, column=3, value=end_info)
    
    end = soup.find_all('br')
    smt = end[15].find_next_sibling(string=True).strip('-')
    address = smt
    split_address = address.split(' - ')
    
    #Municipio
    worksheet.cell(row=linha, column=4, value=split_address[0])
    #UF
    worksheet.cell(row=linha, column=5, value=split_address[1])
    #CEP
    worksheet.cell(row=linha, column=6, value=split_address[2])
    
    
    #Campo muito inconsistente entre os documentos, vale análise futura
    '''end = soup.find(string=' - RESPONSAVEL TÉCNICO IMPLANTAÇÃO (NOME, TELEFONE E E-MAIL):')
    tec = end.find_next_sibling(string=True).strip(' -')
    #Responsáavel Técnico
    worksheet.cell(row=linha, column=7, value=tec)'''
    
    '''end = soup.find(string=' - REPONSÁVEL FINANCEIRO')
    finan = end.find_next_sibling(string=True).strip(' -')
    #Responsável Financeiro
    worksheet.cell(row=linha, column=8, value=finan)'''
    
    #Fazendo que o loop escreva nas linhas corretas
    linha +=1
    
    #Ação para clicar ESC na página
    actions = ActionChains(navegador)
    actions.send_keys(Keys.ESCAPE)
    actions.perform()
    #####################################################################
    time.sleep(2)
    navegador.get("---------------")#link do site (utilizado aqui para voltar para homepage)

In [ ]:
#Criando a planilha de output com as informações
workbook.save("-------------"+ " " + data + ".xlsx")

#Fechando o navegador
navegador.quit()